<a href="https://colab.research.google.com/github/sscherl2017/20Questions/blob/master/lab3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tornado==6

In [2]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs187-2021/lab3-4.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [3]:
# Initialize Otter
import otter
grader = otter.Notebook()

$$
\renewcommand{\mathbf}[1]{\mathbf{#1}}
\renewcommand{\sharp}[1]{\sharp(#1)}
\renewcommand{\underset}[1]{\underset{#1}{\operatorname{argmax}}}{\operatorname{argmax}}
\renewcommand{\operatorname{softmax}}{\operatorname{softmax}}
\renewcommand{\Pr}{\Pr}
\renewcommand{\,|\,}{\,|\,}
$$

# CS187
## Lab 3-4 - Probabilistic parsing and parse disambiguation

Continuing our work on PCFG, you'll implement a probabilistic version of the CKY algorithm and apply it to a parse disambiguation task.

# Preparations {-}

In [4]:
import copy
import math
import nltk
import pandas as pd

from collections import Counter
from collections import defaultdict
from pprint import pprint

# Prepositional phrase attachment ambiguity

In this lab, we'll set aside the rather limited world of arithmetic expressions, focusing on a common example of structural ambiguity in natural language called _prepositional phrase (PP) attachment_. A PP can modify both noun phrases and verb phrases, often creating ambiguity as to what constituent a PP should be attached to.

Here's a small grammar that includes PPs as noun phrase modifiers, as verb phrase modifiers, and as verb arguments.

In [5]:
probabilistic_grammar = nltk.PCFG.fromstring(
    """
    S -> NP VP [1.0]

    NP -> DT N [0.6]

    N -> N PP [0.2]

    VP -> TV NP [0.9] | DTV NP PP [0.1] 

    PP -> P NP [1.0]

    DT ->  'a' [0.5] | 'the' [0.5]

    N -> 'books' [0.2] | 'gifts' [0.2]
    N -> 'table' [0.2] | 'book' [0.2] 

    NP -> 'Twain' [0.2] | 'Howells' [0.2]

    DTV -> 'bought' [0.5] | 'put' [0.5]
    TV ->  'bought' [0.5] | 'saw' [0.5]

    P ->   'on' [0.3] | 'of' [0.4] | 'by' [0.1] | 'for' [0.2]
    """
)

> In this grammar, `TV` stands for "transitive verb", a verb that takes a single NP argument, as in "Twain bought it", and `DTV` stands for "ditransitive verb", which takes two arguments, an NP and a PP, as in "Twain bought it for him".

Notice that the probabilities of all the rules with the same left-hand side sum to 1.

Consider the sentence

In [6]:
example1 = "Twain bought a book for Howells"

Take out a piece of paper and work with your lab partner to draw parse trees for the sentence according to this grammar. How many can you find?

To verify, we'll parse the sentence using a parser provided by NLTK. The `InsideChartParser` returns all parses of a sentence according to a probabilistic grammar along with their probabilities. (It uses a more general algorithm than CKY, so doesn't require the grammar be in CNF.)

Parsing this sentence with the above PCFG results in two possible parses, displaying PP attachment ambiguity:

In [7]:
parser = nltk.parse.InsideChartParser(probabilistic_grammar)
possible_parses = list(parser.parse(example1.split()))

for i, tree in enumerate(possible_parses):
  print(f'Possible parse #{i+1} with probability {tree.prob():.3g}:\n')
  tree.pretty_print()

Possible parse #1 with probability 4.32e-05:

              S                          
   ___________|___                        
  |               VP                     
  |      _________|____                   
  |     |              NP                
  |     |      ________|___               
  |     |     |            N             
  |     |     |    ________|___           
  |     |     |   |            PP        
  |     |     |   |         ___|_____     
  NP    TV    DT  N        P         NP  
  |     |     |   |        |         |    
Twain bought  a  book     for     Howells

Possible parse #2 with probability 2.4e-05:

              S                          
   ___________|_______                    
  |                   VP                 
  |      _____________|________           
  |     |         NP           PP        
  |     |      ___|___      ___|_____     
  NP   DTV    DT      N    P         NP  
  |     |     |       |    |         |    
Twain bought  a 

<!-- BEGIN QUESTION -->

**Question:** What is the more natural parsing, the one that leads to the preferred _reading_ of the sentence, the reading that you understand the sentence as expressing? Is it the most probable parse tree?

<!--
BEGIN QUESTION
name: open_response_pp1
manual: true
-->

The second (DTV) parse leads to the preferred *reading* of the sentence (i.e., the reading that I understand the sentence as expressing). The second parse is *not* the most probable parse tree, however — the first (TV) parse is more probable.

<!-- END QUESTION -->

Change some of the rule probabilities (try to change as few as possible) such that the other tree has higher probability.

<!--
BEGIN QUESTION
name: attach_pp_to_np_1
-->

In [8]:
# TODO - define `probabilistic_grammar_reweighted`.
probabilistic_grammar_reweighted = nltk.PCFG.fromstring(
    """
    S -> NP VP [1.0]

    NP -> DT N [0.6]

    N -> N PP [0.2]

    VP -> TV NP [0.7] | DTV NP PP [0.3] 

    PP -> P NP [1.0]

    DT ->  'a' [0.5] | 'the' [0.5]

    N -> 'books' [0.2] | 'gifts' [0.2]
    N -> 'table' [0.2] | 'book' [0.2] 

    NP -> 'Twain' [0.2] | 'Howells' [0.2]

    DTV -> 'bought' [0.5] | 'put' [0.5]
    TV ->  'bought' [0.5] | 'saw' [0.5]

    P ->   'on' [0.3] | 'of' [0.4] | 'by' [0.01] | 'for' [0.29]
    """
)

In [9]:
parser2 = nltk.parse.InsideChartParser(probabilistic_grammar_reweighted)
possible_parses2 = list(parser2.parse(example1.split()))

for i, tree in enumerate(possible_parses2):
  print(f'Possible parse #{i+1} with probability {tree.prob():.3g}:\n')
  tree.pretty_print()

Possible parse #1 with probability 0.000104:

              S                          
   ___________|_______                    
  |                   VP                 
  |      _____________|________           
  |     |         NP           PP        
  |     |      ___|___      ___|_____     
  NP   DTV    DT      N    P         NP  
  |     |     |       |    |         |    
Twain bought  a      book for     Howells

Possible parse #2 with probability 4.87e-05:

              S                          
   ___________|___                        
  |               VP                     
  |      _________|____                   
  |     |              NP                
  |     |      ________|___               
  |     |     |            N             
  |     |     |    ________|___           
  |     |     |   |            PP        
  |     |     |   |         ___|_____     
  NP    TV    DT  N        P         NP  
  |     |     |   |        |         |    
Twain bought  a

Now we use the PCFG you defined to parse an only slightly different sentence.

In [10]:
example2 = "Twain bought a book by Howells"

possible_parses = list(parser2.parse(example2.split()))

for i, tree in enumerate(possible_parses):
  print('Possible parse #{} with probability {:.3g}:\n'.format(i+1,tree.prob()))
  tree.pretty_print()

Possible parse #1 with probability 3.6e-06:

              S                          
   ___________|_______                    
  |                   VP                 
  |      _____________|________           
  |     |         NP           PP        
  |     |      ___|___      ___|_____     
  NP   DTV    DT      N    P         NP  
  |     |     |       |    |         |    
Twain bought  a      book  by     Howells

Possible parse #2 with probability 1.68e-06:

              S                          
   ___________|___                        
  |               VP                     
  |      _________|____                   
  |     |              NP                
  |     |      ________|___               
  |     |     |            N             
  |     |     |    ________|___           
  |     |     |   |            PP        
  |     |     |   |         ___|_____     
  NP    TV    DT  N        P         NP  
  |     |     |   |        |         |    
Twain bought  a 

<!-- BEGIN QUESTION -->

**Question:** Now what is the more natural parse for the sentence? Is it the most probable one? Can the PCFG be modified such that both sentences are parsed according to the natural readings for these sentences? Try to explain the problem.

<!--
BEGIN QUESTION
name: open_response_attach_pp_to_np_2
manual: true
-->

Now, the second (TV) tree is the more natural parse for the sentence, but the model views the first (DTV) tree as the more probable parse. After some experimentation, it does not seem like the PCFG can be modified such that both sentences are parsed according to the natural readings for these sentences. Ultimately, the sentences seem to be made up of essentially the same constituent rules, save for the use of the DTV vs. TV rule. Thus, the aforementioned rules seems to be the only one that affects the ordering of the most probable tree (i.e., changes to the other rule probabilities don't lead to any differences in overall parse probabilities for the two trees).

<!-- END QUESTION -->

# Conversion to Chomsky Normal Form

The grammar (`probabilistic_grammar`) is almost in Chomsky Normal Form (which would make it suitable for the CKY algorithm). How many of its rules are not in CNF?

<!--
BEGIN QUESTION
name: non_cnf_rule_count
-->

In [11]:
# TODO
non_cnf_rule_count = 1

In [12]:
grader.check("non_cnf_rule_count")

All tests passed!

Convert the grammar `probabilistic_grammar` by hand to a CNF grammar. Try to make as few changes to the grammar as possible.

> **Important:** You should start with `probabilistic_grammar`, not `probabilistic_grammar_reweighted`.

> **Hint:** As part of your solution, you should make use of a new nonterminal `NP_PP`. (Later tests will depend on this.)

<!--
BEGIN QUESTION
name: cnf_conversion
-->

In [13]:
# TODO - convert `probabilistic_grammar` to CNF. You should make
#        use of a new nonterminal `NP_PP`.
probabilistic_grammar_cnf = nltk.PCFG.fromstring(
    """
    S -> NP VP [1.0]

    NP -> DT N [0.6]

    N -> N PP [0.2]

    VP -> TV NP [0.9] | DTV NP_PP [0.1] 

    NP_PP -> NP PP [1.0]

    PP -> P NP [1.0]

    DT ->  'a' [0.5] | 'the' [0.5]

    N -> 'books' [0.2] | 'gifts' [0.2]
    N -> 'table' [0.2] | 'book' [0.2] 

    NP -> 'Twain' [0.2] | 'Howells' [0.2]

    DTV -> 'bought' [0.5] | 'put' [0.5]
    TV ->  'bought' [0.5] | 'saw' [0.5]

    P ->   'on' [0.3] | 'of' [0.4] | 'by' [0.1] | 'for' [0.2]
    """
)

In [14]:
grader.check("cnf_conversion")

All tests passed!

# Probabilistic CKY

You've been availing yourself of a probabilistic parsing algorithm in the NLTK package. It's time to see how such algorithms work. In lab 3-2 you worked with the CKY algorithm as a recognizer and its extension to a parser using backpointers. In the following section you will familiarize yourself with the probabilistic extension of the CKY parser, as presented by Jurafsky & Martin (Chapter 14), which returns the most probable parse (MPP) of a string according to a PCFG grammar.

Now that we have a CNF grammar `probabilistic_grammar_cnf`, we can use the CKY algorithm to parse an example sentence. For reference, here is a pseudo-code version of the probabilistic CKY algorithm:

```
 1.  define cky-mpp(string = w1, ..., wN, grammar):
 2.      for j in [1..N]:                     # each end string position

             # handle rules of the form A -> w
 3.          for all A where A -> wj in grammar:
 4.              T[j-1, j, A] := Pr(A -> wj)

             # handle rules of the form A -> B C
 5.          for length in [2..j]:            # each subconstituent length
 6.              i := j - length              # start string position
 7.              for split in [i+1..j-1]      # each split point
 8.                  for all A where 
 9.                          A -> B C in grammar
10.                          and T[i, split, B] > 0
11.                          and T[split, j, C] > 0:
12.                      new_prob := Pr(A -> B C)
13.                                  x table[i, split, B]
14.                                  x table[split, j, C]
15.                      if T[i, j, A] < new_prob
16.                         then T[i, j, A] := new_prob
17,                              back[i, j, A] := (split, B, C)
18.      return (build_tree(back[0, N, S]), T[0, N, S]
```

This PCKY algorithm is almost identical to the CKY variant you used in lab 3-2, with only a few differences, namely:

1. Table dimensions for a sentence of $N$ words:

  * CKY: $(N+1)\times(N+1)$
  
  * PCKY: $(N+1)\times(N+1)\times |\cal{N}|$

2. Table values:

  * CKY: list of constituents
  
  * PCKY: probabilities, where `table[i, j, A]` is the maximum probability of nonterminal `A` covering words between string positions `i` and `j`

3. Backpointers:

  * CKY: mapping from nonterminals to set of all possible split positions and rules
  
  * PCKY: mapping from nonterminals to the single most probable split position and rule

Notice that the probabilities of all the rules with the same left-hand side sum to 1.

We will implement the required $(N+1)\times(N+1)\times|\cal{N}|$ three-dimensional tables as $(N+1)\times(N+1)$ two-dimensional tables, in which each cell will hold a dictionary mapping nonterminals to the required entry values. We will implement separate recognition and backpointer tables:

* For the recognition table `table`, the entry values are the probabilities.
* For the backpointers table `back`, the entry values are the appropriate backpointer `(split, B, C)`.

As in lab 3-2, all the cells that need not be filled contain '---'. All other cells are initialized with an appropriate default dictionary. Run the following code to initialize the tables. (You don't need to go over it, we will look at a specific cell to better understand the content.)

In [15]:
def make_tables(words):
    words = [""] + words
    N = len(words)

    # initialize data in tables
    table_data = [["---" for i in range(N)] for j in range(N)]
    back_data = [["---" for i in range(N)] for j in range(N)]

    # add in upper triangular elements
    for i in range(N):
        for j in range(N):
            if i < j:
                table_data[i][j] = defaultdict(float)
                back_data[i][j] = defaultdict(lambda x: None)

    # generate corresponding data frames
    table = pd.DataFrame(table_data, columns=words, index=range(N))
    table.columns = pd.MultiIndex.from_arrays([table.columns] + [range(N)])
    back = pd.DataFrame(back_data, columns=words, index=list(range(N)))
    back.columns = pd.MultiIndex.from_arrays([back.columns] + [range(N)])
    return (table, back)
    
table, back = make_tables(example1.split())

Let's print out one of the tables:

In [16]:
table

,,Twain,bought,a,book,for,Howells
,0,1,2,3,4,5,6
0,---,{},{},{},{},{},{}
1,---,---,{},{},{},{},{}
2,---,---,---,{},{},{},{}
3,---,---,---,---,{},{},{}
4,---,---,---,---,---,{},{}
5,---,---,---,---,---,---,{}
6,---,---,---,---,---,---,---


We'll "play parser" and fill both tables for the first four values of `j` following the pseudo-code above. Below, you'll finish filling the tables for the remaining two values of `j` yourself. Make sure you understand what's going on in the cell below before continuing.

In [17]:
                              # j = 1 (Twain)
table.iloc[0,1]['NP'] = 0.2

                              # j = 2 (bought)
table.iloc[1,2]['DTV'] = 0.5
table.iloc[1,2]['TV'] = 0.5
                                # i = 0, split = 1: no changes

                              # j = 3 (a)
table.iloc[2,3]['DT'] = 0.5
                                # i = 1, split = 2: no changes
                                # i = 0, split = 1: no changes
                                # i = 0, split = 2: no changes

                              # j = 4 (book)
table.iloc[3,4]['N'] = 0.2
                                # i = 2, split = 3
table.iloc[2,4]['NP'] = 0.06
back.iloc[2,4]['NP'] = (3, 'DT', 'N')
                                # i = 1, split = 2
table.iloc[1,4]['VP'] = 0.027
back.iloc[1,4]['VP'] = (2, 'TV', 'NP')
                                # i = 1, split = 3: no changes
                                # i = 0, split = 1
table.iloc[0,4]['S'] = 0.0054
back.iloc[0,4]['S'] = (1, 'NP', 'VP')
                                # i = 0, split = 2: no changes
                                # i = 0, split = 3: no changes

We can see what progress has been made so far by examining the parser table.

In [22]:
table

,,Twain,bought,a,book,for,Howells
,0,1,2,3,4,5,6
0,---,{'NP': 0.2},{},{},{'S': 0.0054},{},{'S': 4.320000000000001e-05}
1,---,---,"{'DTV': 0.5, 'TV': 0.5}",{},{'VP': 0.027},{},{'VP': 0.00021600000000000005}
2,---,---,---,{'DT': 0.5},{'NP': 0.06},{},{'NP': 0.00048}
3,---,---,---,---,{'N': 0.2},{},{'N': 0.0016000000000000003}
4,---,---,---,---,---,{'P': 0.2},{'PP': 0.04000000000000001}
5,---,---,---,---,---,---,{'NP': 0.2}
6,---,---,---,---,---,---,---


Now it's time for you to finish filling the last two columns of the table.

<!--
BEGIN QUESTION
name: pcky_fill_table
points: 6
-->

In [19]:
# TODO - Continue filling in the tables as shown above.
#        *DO NOT* round any results since hidden tests check for them.
table.iloc[0,6]['S'] = 1 * 0.2 * 0.9 * 0.5 * 0.6 * 0.5 * 0.2 * 0.2 * 0.04
back.iloc[0,6]['S'] = (1, 'NP', 'VP')

table.iloc[1,6]['VP'] = 0.9 * 0.5 * 0.6 * 0.5 * 0.2 * 0.2 * 0.04
back.iloc[1,6]['VP'] = (2, 'TV', 'NP')

table.iloc[2,6]['NP'] = 0.6 * 0.5 * 0.2 * 0.2 * 0.04
back.iloc[2,6]['NP'] = (3, 'DT', 'N')

table.iloc[3,6]['N'] = 0.2 * 0.2 * 0.04
back.iloc[3,6]['N'] = (4, 'N', 'PP')

table.iloc[4,6]['PP'] = 0.2 * 0.2
back.iloc[4,6]['PP'] = (5, 'P', 'NP')

table.iloc[5,6]['NP'] = 0.2

table.iloc[4,5]['P'] = 0.2

In [20]:
grader.check("pcky_fill_table")

All tests passed!

In addition to the table of probabilities, the algorithm also maintains the backpointer table.

In [21]:
back

,,Twain,bought,a,book,for,Howells
,0,1,2,3,4,5,6
0,---,{},{},{},"{'S': (1, 'NP', 'VP')}",{},"{'S': (1, 'NP', 'VP')}"
1,---,---,{},{},"{'VP': (2, 'TV', 'NP')}",{},"{'VP': (2, 'TV', 'NP')}"
2,---,---,---,{},"{'NP': (3, 'DT', 'N')}",{},"{'NP': (3, 'DT', 'N')}"
3,---,---,---,---,{},{},"{'N': (4, 'N', 'PP')}"
4,---,---,---,---,---,{},"{'PP': (5, 'P', 'NP')}"
5,---,---,---,---,---,---,{}
6,---,---,---,---,---,---,---


Reconstruct the parse tree that the algorithm finds by chasing backpointers in this table, and enter it using parenthesized string notation in the next cell.

> **Hint:** You'll definitely need pencil and paper for this. Start with the entry at 0,6 and work backwards from there.

<!--
BEGIN QUESTION
name: best_parse
-->

In [ ]:
# TODO
best_parse = nltk.Tree.fromstring(
    "(S (NP Twain) (VP (TV bought) (NP (DT a) (N (N book) (PP (P for) (NP Howells))))))"
)
best_parse.pretty_print()

              S                          
   ___________|___                        
  |               VP                     
  |      _________|____                   
  |     |              NP                
  |     |      ________|___               
  |     |     |            N             
  |     |     |    ________|___           
  |     |     |   |            PP        
  |     |     |   |         ___|_____     
  NP    TV    DT  N        P         NP  
  |     |     |   |        |         |    
Twain bought  a  book     for     Howells



In [ ]:
grader.check("best_parse")

All tests passed!

Let us look at the probability of this tree, as cached in the [0,6] entry in the probability table:

In [ ]:
table.iloc[0,6]

defaultdict(float, {'S': 4.320000000000001e-05})

<!-- BEGIN QUESTION -->

**Question:** Compare the probability of this tree you just computed to the probabilities of the parses that the NLTK algorithm generated at the start of the lab, and explain the result.

<!--
BEGIN QUESTION
name: open_response_pcky
manual: true
-->

The computed parse tree and probability of that tree matches the probability of the first (most probable) parse tree that the NLTK algorithm generated at the start of the lab. Thus, the CKY algorihm correctly found the most probable tree and correctly computed this probability.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Lab debrief – for consensus submission only

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on might include the following: 

* Was the lab too long or too short?
* Were the readings appropriate for the lab? 
* Was it clear (at least after you completed the lab) what the points of the exercises were? 
* Are there additions or changes you think would make the lab better?

but you should comment on whatever aspects you found especially positive or negative.

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

Overall, the lab was a little too long – we had to spend time out of class finishing it up. However, the readings were appropriate, and the points of the exercises were clear (after we completed the labs). No other changes to report.

<!-- END QUESTION -->



# End of Lab 3-4 {-}

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

best_parse:

    All tests passed!
    

cnf_conversion:

    All tests passed!
    

non_cnf_rule_count:

    All tests passed!
    

pcky_fill_table:

    All tests passed!